In [9]:
#from dotenv import find_dotenv, load_dotenv
#OPENAI_API_KEY=''
import openai
import os
openai.api_key  = os.getenv(OPENAI_API_KEY)
#dotenv_path = find_dotenv()
#load_dotenv(dotenv_path)

In [7]:
from langchain.agents.react.base import DocstoreExplorer
from langchain.docstore import Wikipedia
docstore = DocstoreExplorer(Wikipedia())

/home/sage/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.react.base.DocstoreExplorer` was deprecated in langchain 0.1.0 and will be removed in 0.2.0
  warn_deprecated(


In [8]:
from langchain.agents import Tool 
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search",
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup",
    ),
]

In [9]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4", openai_api_key="")


In [10]:
from langchain.agents import AgentType, create_react_agent
react = initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True, handle_parsing_errors=True)

In [11]:
question = "Who is elon musk"
result = react.invoke(question)



> Entering new AgentExecutor chain...
Thought: I need to search Elon Musk and describe who he is.
Action: Search[Elon Musk]
Observation: Could not find [Elon Musk]. Similar: ['Elon Musk', 'Acquisition of Twitter by Elon Musk', 'Twitter under Elon Musk', 'Views of Elon Musk', 'Musk family', 'Wealth of Elon Musk', "Elon Musk's Tesla Roadster", 'Maye Musk', 'Elon Musk vs. Mark Zuckerberg', 'Elon Musk (Isaacson book)']
Thought:I can search the first suggestion which is 'Elon Musk'.
Action: Search[Elon Musk]
Observation: Could not find [Elon Musk]. Similar: ['Elon Musk', 'Acquisition of Twitter by Elon Musk', 'Twitter under Elon Musk', 'Views of Elon Musk', 'Musk family', 'Wealth of Elon Musk', "Elon Musk's Tesla Roadster", 'Maye Musk', 'Elon Musk vs. Mark Zuckerberg', 'Elon Musk (Isaacson book)']
Thought:The search function is not returning the expected results for Elon Musk. I already know that Elon Musk is a prominent entrepreneur and business magnate, known for founding SpaceX and co-

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

#which tools will be usefull for (a) Get latest stock price from Yahoo Finance (b) Get latest company news links/summaries from FinViz
#(c) Get latest 10-K information (summarized) from SEC)
load_tools([])

In [66]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [67]:
from langchain.agents import AgentType, initialize_agent
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.0)
tools = [YahooFinanceNewsTool()]
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [69]:
agent_chain.invoke(
    "what did microsoft close at in the previous week the ticker for microsoft is MSFT?",
)



> Entering new AgentExecutor chain...
I should check the closing price for Microsoft in the previous week to answer this question.
Action: yahoo_finance_news
Action Input: MSFT
Observation: Microsoft Corporation (MSFT) Expanded Beyond Historical Core
Meridian Funds, managed by ArrowMark Partners, released its “Meridian Hedged Equity Fund” fourth quarter 2023 investor letter. A copy of the same can be downloaded here. News of declining inflation and the potential for lower interest rates in 2024 fueled optimism in the final months of the year. Equities saw a broad rally, with small […]
Thought:This news article does not provide the closing price for Microsoft in the previous week. I should try again.
Action: yahoo_finance_news
Action Input: MSFT
Observation: Microsoft Corporation (MSFT) Expanded Beyond Historical Core
Meridian Funds, managed by ArrowMark Partners, released its “Meridian Hedged Equity Fund” fourth quarter 2023 investor letter. A copy of the same can be downloaded here.

KeyboardInterrupt: 

In [52]:
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
from langchain.chains import create_extraction_chain

In [35]:
async def run_playwright(site):
    data = ""
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)

        page = await browser.new_page()
        await page.goto(site)

        page_source = await page.content()
        soup = BeautifulSoup(page_source, "html.parser")
        
        for script in soup(["script", "style"]): # remove all javascript and stylesheet code
            script.extract()
        # get text
        text = soup.get_text()
        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        data = '\n'.join(chunk for chunk in chunks if chunk)

        await browser.close()
    return data

output = await run_playwright("https://finviz.com/news.ashx")

In [36]:
output

'Stock Market News & Blogs\nSubscribe to our newsletter to receive the latest product updates and special offers\nSubscribe\nDismiss\nHomeNewsScreenerMapsGroupsPortfolioInsiderFuturesForexCryptoBacktestsEliteWed APR 03 2024 2:17 PM ET\nTheme\nHelp\nLogin\nRegister\nNews\nBlogs\nView by\nTime\nSource\n02:15PM\nGold plows to record high after Powell\'s remarks\n02:11PM\n‘Please don’t judge me’: I’m 54, and married with 5 kids. I have $20,000 in debt and $20,000 in mutual funds. I just inherited $10,000. How do I invest it?\n02:04PM\nStock Market Today: S&P 500 Rises After Powell Reassures on Rate Cuts\n01:59PM\nOptions traders see rising risk of Fed rate hike following oil rally, solid U.S. data\n01:57PM\nVenture Firm Drive Capital Nears Stake Sale to Collective Global\n01:51PM\nLoading…\n01:51PM\nThe company he keeps: How an investor got nabbed insider trading on Trump’s ‘DJT’ stock merger\n01:50PM\nPowell says supply side may continue to help bring down inflation this year\n01:50PM\nWh

In [37]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain

llm = ChatOpenAI(temperature=0, model="gpt-4", openai_api_key='')
structured_schema = {
    "properties": {
        "url": {"type":"string"}
    },
}
extraction_chain = create_extraction_chain(structured_schema, llm)
extraction_chain.run(output)

/home/sage/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/home/sage/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


[{'url': 'https://www.finviz.com/'}]

In [49]:
from langchain_community.document_loaders import AsyncHtmlLoader

urls = ["https://finviz.com/news.ashx"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()

Fetching pages: 100%|#####################################################################| 1/1 [00:00<00:00,  6.12it/s]


In [50]:
docs

[Document(page_content='<!DOCTYPE html>\n<html lang="en" class=" dark">\n<head>\n<title>Stock Market News & Blogs</title>\n<meta charset="UTF-8"><meta name="viewport" content="width=1024"><meta name="description" content="Stock screener for investors and traders, financial visualizations.">\n<meta http-equiv="Refresh" content="3600">\n\r\n            <link rel="preload" href="/fonts/lato-v17-latin-ext_latin-regular.woff2" as="font" crossorigin>\r\n            <link rel="preload" href="/fonts/lato-v17-latin-ext_latin-700.woff2" as="font" crossorigin>\r\n            <link rel="preload" href="/fonts/lato-v17-latin-ext_latin-900.woff2" as="font" crossorigin>\r\n            <link rel="preload" href="/fonts/inter-latin.woff2" as="font" crossorigin>\r\n        \r\n            <script>\r\n                window.notificationsArray = [];\r\n                window.renderScriptNotLoaded = function () {};\r\n                window.handleScriptNotLoaded = function (element) {\r\n                    

In [51]:
# Transform
bs_transformer = BeautifulSoupTransformer()
docs_transformed = bs_transformer.transform_documents(docs, tags_to_extract=["a", "href"])

NameError: name 'BeautifulSoupTransformer' is not defined

In [65]:
# Result
#.page_content[0:500]
docs_transformed[0]

Document(page_content="Home (/) News (/news.ashx) Screener (/screener.ashx) Maps (/map.ashx) Groups (/groups.ashx) Portfolio (/portfolio.ashx) Insider (/insidertrading.ashx) Futures (/futures.ashx) Forex (/forex.ashx) Crypto (/crypto.ashx) Backtests (/elite.ashx?utm_source=finviz&utm_medium=banner&utm_campaign=main-navbar-backtests) Elite (/elite.ashx)  (#)        (#) Theme  (#) Help (/help/screener.ashx) Login (/login.ashx) Register (/register.ashx) Time (news.ashx) Source (news.ashx?v=2) Traders Scramble to Bet Against Trump Media (https://www.nytimes.com/2024/04/03/business/trump-media-short-selling.html) Disney, Iger prevail over activist Peltz in proxy fight (https://finance.yahoo.com/news/disney-defeats-activist-investor-nelson-peltz-in-proxy-fight-172400941.html) Hunt for Iger's successor now kicks into high gear (https://finance.yahoo.com/news/analysis-hunt-disney-ceo-bob-173144766.html) We don't have a tax collection problem, we have a government spending problem: Adam Johnson

In [11]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(openai_api_key="")
#this is one way of prompting your llm however a better way is to create a prompt template
#llm.invoke("how can langsmith help with testing")
#Here is our prompt template 
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

#This is a chain
chain = prompt | llm 
#After you have created the chain you can now prompt the llm this way.
chain.invoke({"input": "how can langsmith help with testing"})

AIMessage(content="Langsmith is a powerful tool that can greatly assist with various aspects of testing in software development. Here are some ways in which Langsmith can help with testing:\n\n1. **Automated Testing**: Langsmith can be used to automate the testing process, allowing developers to write test cases in a structured way using the language's syntax. This can help in quickly running tests and identifying any bugs or issues in the code.\n\n2. **Test Data Generation**: Langsmith can generate test data dynamically, making it easier to cover various scenarios and edge cases during testing. This can help in improving test coverage and ensuring the robustness of the software.\n\n3. **Integration Testing**: Langsmith can be used to write integration tests that cover the interaction between different components of the software. This can help in identifying any issues that arise when different parts of the system are combined.\n\n4. **Performance Testing**: Langsmith can assist in wri

In [14]:
#the output of the above answer is a message and we may need to convert it to a string for other usecases.
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

#now add it to our chain
chain = prompt | llm | output_parser
#now rather than a chatMessage we get a string back
chain.invoke({"input": "how can langsmith help with testing?"})

"Langsmith can help with testing in several ways, including:\n\n1. Test Automation: Langsmith can automate the process of testing software by generating test scripts and executing test cases. This can help reduce manual testing efforts and increase the efficiency of the testing process.\n\n2. Test Data Generation: Langsmith can generate realistic and diverse test data sets that can be used to test the software under various conditions. This can help uncover potential issues and ensure the software's robustness.\n\n3. Code Coverage Analysis: Langsmith can analyze code coverage to determine which parts of the software have been tested and which parts have not. This can help identify areas that require additional testing to ensure comprehensive test coverage.\n\n4. Performance Testing: Langsmith can be used to simulate different user loads and scenarios to test the performance of the software. This can help identify bottlenecks and performance issues that need to be addressed.\n\n5. Integ

In [17]:
# We need to add aditional context to appropriatly answer our question becuase the model(llm) is based on 2022.
# When we have to much data to pass to the llm directly we use retrieval
# A retriever will fetch only the most relevant data and pass those in
# It can use whatever source you want DB's, internet etc
# need to read on vetctor stores...................
# Here we will populate a vector store and use it as a retriever

# Load data that we want to index with a WebBaseLoader, pip install beautifulsoup4
# import WebBaseLoader
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()

#index the info from the URL into our vector store, for this we need an embedding model and a vectorstore.
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key="")

#now use this embedding model to ingest documents into a vectorstore. use a simple local vectorstore, FAISS.
#install FAISS- pip install faill-cpu
#build index
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

#now the documents from the URL have been loaded,embedded, and indexed into our vetor store.
# what is the text splitter?.....................................

In [18]:
#Now that we have this data indexed in a vectorstore, we will create a retrieval chain. 
#This chain will take an incoming question, look up relevant documents, 
#then pass those documents along with the original question into an LLM and ask it to answer the original question.


# set up the chain that takes a question and the retrieved documents and generates an answer.
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [19]:
#If we wanted to, we could run this ourselves by passing in documents directly:

# from langchain_core.documents import Document

# document_chain.invoke({
#     "input": "how can langsmith help with testing?",
#     "context": [Document(page_content="langsmith can let you visualize test results")]
# })

#However we want the documents to first come from the retriever we just set up. That way we can use the retriever to dynamically select the most
# relevant documents and pass those in for a given question.

from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)


In [20]:
#We can now invoke this chain. This returns a dictionary - the response from the LLM is in the answer key
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

LangSmith can help with testing by allowing developers to create datasets for running tests on LLM applications, uploading test cases in bulk or creating them on the fly, exporting test cases from application traces, running custom evaluations, comparing different versions of applications, providing a playground environment for rapid iteration and experimentation, enabling beta testing to collect real-world performance data, capturing feedback from users, annotating traces for evaluation, adding runs to datasets to refine performance, and providing monitoring and A/B testing capabilities.


In [21]:
#This next section will be about agents
# let's set up a tool for the retriever we just created

from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [37]:
# Now a pre built tool that is a search engine, Tavily
# TAVILY_API_KEY = ""
# tavily_api_key=""
os.environ["TAVILY_API_KEY"] = ""
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(tavily_api_key="")


In [38]:
# Now create a list of tools to work with 
tools = [retriever_tool, search]

In [40]:
# now we use it to get a pre defined promt

from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# You need to set OPENAI_API_KEY environment variable or pass it as argument `openai_api_key`.
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key="")
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [41]:
# now lets ask it some questions.
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricingSelf-HostingTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping​Prototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.
The ability to rapidly understand how the model is performing — and de

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith is a platform for Language Learning Model (LLM) application development, monitoring, and testing. Here are some ways it can help with testing:\n\n1. **Debugging**: LangSmith provides clear visibility and debugging information at each step of an LLM sequence, making it easier to identify and root-cause issues.\n\n2. **Initial Test Set**: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications. These test cases can be uploaded in bulk, created on the fly, or exported from application traces.\n\n3. **Comparison View**: When prototyping different versions of your applications and making changes, LangSmith provides a user-friendly comparison view for test runs to track and diagnose regressions in test scores across multiple revisions of your application.\n\n4. **Playground**: LangSmith provides a playground environment fo

In [42]:
agent_executor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'current weather in San Francisco'}`


[{'url': 'https://www.weatherapi.com/', 'content': "Weather in San Francisco is {'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1712276648, 'localtime': '2024-04-04 17:24'}, 'current': {'last_updated_epoch': 1712276100, 'last_updated': '2024-04-04 17:15', 'temp_c': 10.6, 'temp_f': 51.1, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 18.6, 'wind_kph': 29.9, 'wind_degree': 180, 'wind_dir': 'S', 'pressure_mb': 1006.0, 'pressure_in': 29.72, 'precip_mm': 0.49, 'precip_in': 0.02, 'humidity': 56, 'cloud': 100, 'feelslike_c': 7.8, 'feelslike_f': 46.0, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 2.0, 'gust_mph': 19.5, 'gust_kph': 31.3}}"}, {'url': 'h

{'input': 'what is the weather in SF?',
 'output': 'The current weather in San Francisco, California is overcast with a temperature of 10.6°C (51.1°F). The wind is coming from the south at a speed of 18.6 mph. The humidity is at 56% and there is a visibility of 16 km. Please note that the weather conditions can change rapidly.'}

In [45]:
#creating the base for our project with a pre built yahoo finance tool.
#Now I need to make a tool for finviz which will take more work due to no pre built options.
#Also some sort of promt is needed for the agent to utilize.

#I need to checkout playwright becuase next I want to build out the finviz tool that goes to a page and then navigates to url's from that page.
#Now I believe the above step cannot be done in a single step....YET. Unless I am unaware of a tool for this??

from langchain.agents import AgentType, create_react_agent
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.0, openai_api_key="")
tools = [YahooFinanceNewsTool()]
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [5]:
# this is the start of my finviz tool
# It shoult scrape finviz, return links(and possibly descriptions for the llm to reason over which link to choose accoridng to the user input)
from langchain_community.document_loaders import AsyncHtmlLoader

urls = ["https://finviz.com/news.ashx"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()
docs

Fetching pages: 100%|#####################################################################| 1/1 [00:00<00:00,  8.21it/s]


[Document(page_content='<!DOCTYPE html>\n<html lang="en" class=" dark">\n<head>\n<title>Stock Market News & Blogs</title>\n<meta charset="UTF-8"><meta name="viewport" content="width=1024"><meta name="description" content="Stock screener for investors and traders, financial visualizations.">\n<meta http-equiv="Refresh" content="3600">\n\r\n            <link rel="preload" href="/fonts/lato-v17-latin-ext_latin-regular.woff2" as="font" crossorigin>\r\n            <link rel="preload" href="/fonts/lato-v17-latin-ext_latin-700.woff2" as="font" crossorigin>\r\n            <link rel="preload" href="/fonts/lato-v17-latin-ext_latin-900.woff2" as="font" crossorigin>\r\n            <link rel="preload" href="/fonts/inter-latin.woff2" as="font" crossorigin>\r\n        \r\n            <script>\r\n                window.notificationsArray = [];\r\n                window.renderScriptNotLoaded = function () {};\r\n                window.handleScriptNotLoaded = function (element) {\r\n                    

In [6]:
# Transform
from langchain_community.document_transformers import BeautifulSoupTransformer
bs_transformer = BeautifulSoupTransformer()
docs_transformed = bs_transformer.transform_documents(docs, tags_to_extract=["a", "href"])
# Result
#.page_content[0:500]
docs_transformed[0]

#MY thoughts on this are that the data is broken down enough for the llm to read the headline and/or the url and decide which url to navigate to and
#scrape for the next step.

Document(page_content='Home (/) News (/news.ashx) Screener (/screener.ashx) Maps (/map.ashx) Groups (/groups.ashx) Portfolio (/portfolio.ashx) Insider (/insidertrading.ashx) Futures (/futures.ashx) Forex (/forex.ashx) Crypto (/crypto.ashx) Backtests (/elite.ashx?utm_source=finviz&utm_medium=banner&utm_campaign=main-navbar-backtests) Elite (/elite.ashx)  (#)        (#) Theme  (#) Help (/help/screener.ashx) Login (/login.ashx) Register (/register.ashx) Time (news.ashx) Source (news.ashx?v=2) Wall St Week Ahead US stocks\' lofty valuations in spotlight as earnings season nears (https://www.reuters.com/markets/us/wall-st-week-ahead-us-stocks-lofty-valuations-spotlight-earnings-season-nears-2024-04-05/) ‘Please don’t judge me’: I’m 54, and married with 5 kids. I have $20,000 in debt and $20,000 in mutual funds. I just inherited $10,000. How do I invest it? (https://www.marketwatch.com/story/please-dont-judge-me-im-54-and-married-with-5-kids-i-have-20-000-in-debt-and-20-000-in-mutual-funds-i

In [8]:
#I have the data how I want it now. I need to load it into a vector store so the llm can reason over which link to navigate to next
#This would include vectorizing, storing, and the llm retreiving the data afterward
#Or is there a way to pass this data directly to my llm now and allow it to reason, navigate, summarize.
#the question is, do I need to vectorize this ammount of data? I THINK

#im going to start vectorizing what I have and seeing what will happen....

#index the info from the URL into our vector store, for this we need an embedding model and a vectorstore.
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key="")

#now use this embedding model to ingest documents into a vectorstore. use a simple local vectorstore, FAISS.
#install FAISS- pip install faill-cpu
#build index
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs_transformed)
vector = FAISS.from_documents(documents, embeddings)

#now the documents from the URL have been loaded,embedded, and indexed into our vetor store.

In [14]:
# set up the chain that takes a question and the retrieved documents and generates an answer.
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4", openai_api_key='')

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [15]:
#we want the documents to first come from the retriever we just set up. That way we can use the retriever to dynamically select the most
# relevant documents and pass those in for a given question.

from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [16]:
#We can now invoke this chain. This returns a dictionary - the response from the LLM is in the answer key
response = retrieval_chain.invoke({"input": "is there anything happening with cannabis recently?"})
print(response["answer"])

Yes, Leafly won a cannabis advertising case as a New York court struck down a pot-marketing ban.
